**Подготовка окружения**

Установка библиотеки, осуществляющей расчёт коэффициентов по RR-интервалам.

In [1]:
! pip install hrv-analysis
! pip install nolds

In [54]:
# Загрузка функций для анализа
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
from hrvanalysis import get_frequency_domain_features
from hrvanalysis import get_poincare_plot_features
from hrvanalysis import get_sampen
from scipy.stats import kurtosis
from scipy.stats import skew
import numpy as np
import pandas as pd

In [55]:
# Изменение библиотечной функции get_time_domain_features
from typing import List, Tuple
def get_time_domain_features(nn_intervals: List[float], pnni_as_percent: bool = True) -> dict:
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals) - 1 if pnni_as_percent else len(nn_intervals)

    # Basic statistics
    mean_nni = np.mean(nn_intervals)
    median_nni = np.median(nn_intervals)
    range_nni = max(nn_intervals) - min(nn_intervals)

    sdsd = np.std(diff_nni)
    rmssd = np.sqrt(np.mean(diff_nni ** 2))

    nni_50 = sum(np.abs(diff_nni) > 50)
    pnni_50 = 100 * nni_50 / length_int
    nni_25 = sum(np.abs(diff_nni) > 20)
    pnni_25 = 100 * nni_25 / length_int

    # Feature found on github and not in documentation
    cvsd = rmssd / mean_nni

    # Features only for long term recordings
    sdnn = np.std(nn_intervals, ddof=1)  # ddof = 1 : unbiased estimator => divide std by n-1
    cvnni = sdnn / mean_nni

    # Heart Rate equivalent features
    heart_rate_list = np.divide(60000, nn_intervals)
    mean_hr = np.mean(heart_rate_list)
    min_hr = min(heart_rate_list)
    max_hr = max(heart_rate_list)
    std_hr = np.std(heart_rate_list)

    time_domain_features = {
        'mean_nni': mean_nni,
        'sdnn': sdnn,
        'sdsd': sdsd,
        'nni_50': nni_50,
        'pnni_50': pnni_50,
        'nni_25': nni_25,
        'pnni_25': pnni_25,
        'rmssd': rmssd,
        'median_nni': median_nni,
        'range_nni': range_nni,
        'cvsd': cvsd,
        'cvnni': cvnni,
        'mean_hr': mean_hr,
        "max_hr": max_hr,
        "min_hr": min_hr,
        "std_hr": std_hr,
    }

    return time_domain_features

In [56]:
# Подключение гугл-диска, на котором хранятся данные
! pip install -q kaggle
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [109]:
# Десериализация полученной модели
import pickle

file = "adaboostpickled.tmp"
with open("/content/drive/MyDrive/Colab Notebooks/Data_HRV/WESAD_and_SWELL/" + file, "rb") as infile:
    pickled_model = pickle.load(infile)

**Загрузка данных**

In [151]:
# Загрузка txt файла из моих данных
data_list = list(np.loadtxt("1.2.txt"))
# Define the time interval in milliseconds (1 minute = 60000 milliseconds)
interval = 60000
# Initialize variables
one_minute_intervals = []
current_interval = []
current_sum = 0

# Iterate through the data
for value in data_list:
    # Add the current value to the current interval
    current_interval.append(value)
    current_sum += value
    # If the sum of the current interval is greater than or equal to 1 minute, add it to the list of one-minute intervals and reset the current interval
    if current_sum >= interval:

        one_minute_intervals.append(current_interval)
        current_interval = []
        current_sum = 0
for i in one_minute_intervals:
  print(sum(i))
  print(sum(i)/len(i))
  print(i)

60458.0
785.1688311688312
[781.0, 781.0, 734.0, 750.0, 734.0, 719.0, 797.0, 844.0, 781.0, 766.0, 781.0, 734.0, 750.0, 734.0, 688.0, 688.0, 672.0, 703.0, 719.0, 703.0, 734.0, 766.0, 734.0, 719.0, 766.0, 734.0, 703.0, 688.0, 703.0, 672.0, 750.0, 781.0, 813.0, 891.0, 938.0, 922.0, 859.0, 891.0, 859.0, 813.0, 844.0, 859.0, 859.0, 797.0, 828.0, 844.0, 875.0, 813.0, 906.0, 891.0, 875.0, 875.0, 969.0, 844.0, 766.0, 813.0, 734.0, 766.0, 719.0, 781.0, 813.0, 766.0, 797.0, 797.0, 781.0, 797.0, 844.0, 781.0, 766.0, 734.0, 766.0, 797.0, 719.0, 766.0, 766.0, 781.0, 734.0]
60007.0
789.5657894736842
[797.0, 828.0, 859.0, 875.0, 891.0, 875.0, 844.0, 844.0, 781.0, 828.0, 844.0, 813.0, 859.0, 797.0, 797.0, 844.0, 938.0, 688.0, 750.0, 703.0, 719.0, 703.0, 734.0, 797.0, 750.0, 719.0, 734.0, 766.0, 766.0, 828.0, 734.0, 734.0, 719.0, 719.0, 750.0, 859.0, 906.0, 750.0, 844.0, 828.0, 828.0, 844.0, 813.0, 766.0, 766.0, 719.0, 672.0, 656.0, 781.0, 750.0, 813.0, 594.0, 750.0, 781.0, 844.0, 813.0, 891.0, 875.0, 7

In [153]:
# Загрузка из csv для WESAD
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data_HRV/wesad/S2_E4_Data/IBI.csv")
df_data = df.iloc[:, -1]
df_data.iloc[:] = df_data.iloc[:]*1000 # Change to milliseconds
df_data.head()

# Обработка для WESAD
data_list = df_data.tolist()

# Define the time interval in milliseconds (1 minute = 60000 milliseconds)
interval = 60000

# Initialize variables
one_minute_intervals = []
current_interval = []
current_sum = 0

# Iterate through the data
for value in data_list:
    # Add the current value to the current interval
    current_interval.append(value)
    current_sum += value
    # If the sum of the current interval is greater than or equal to 1 minute, add it to the list of one-minute intervals and reset the current interval
    if current_sum >= interval:
        one_minute_intervals.append(current_interval)
        current_interval = []
        current_sum = 0
print(one_minute_intervals)

[[765.66, 890.6659999999999, 781.2860000000001, 812.5369999999999, 781.2860000000001, 703.157, 734.409, 843.789, 750.034, 796.9110000000001, 734.409, 859.414, 671.906, 640.654, 703.157, 687.531, 734.409, 828.163, 765.66, 765.66, 765.66, 734.409, 609.403, 765.66, 828.163, 718.7829999999999, 640.654, 656.28, 593.777, 609.403, 656.28, 718.7829999999999, 796.9110000000001, 781.2860000000001, 718.7829999999999, 765.66, 734.409, 718.7829999999999, 671.906, 765.66, 796.9110000000001, 859.414, 859.414, 859.414, 828.163, 796.9110000000001, 750.034, 781.2860000000001, 750.034, 796.9110000000001, 796.9110000000001, 843.789, 796.9110000000001, 765.66, 812.5369999999999, 750.034, 765.66, 718.7829999999999, 750.034, 781.2860000000001, 796.9110000000001, 859.414, 828.163, 765.66, 750.034, 796.9110000000001, 828.163, 828.163, 796.9110000000001, 796.9110000000001, 734.409, 765.66, 750.034, 703.157, 734.409, 734.409, 796.9110000000001, 890.6659999999999, 859.414], [843.789, 875.0400000000001, 796.911000

In [104]:
# Загрузка txt файла для PhysioNet
data_list = list(np.loadtxt("/content/drive/MyDrive/Colab Notebooks/Data_HRV/PhysioNet_healthy_subjects/009.txt"))
# Define the time interval in milliseconds (1 minute = 60000 milliseconds)
interval = 60000

# Initialize variables
one_minute_intervals = []
current_interval = []
current_sum = 0

# Iterate through the data
for value in data_list:
    # Add the current value to the current interval
    current_interval.append(value)
    current_sum += value
    # If the sum of the current interval is greater than or equal to 1 minute, add it to the list of one-minute intervals and reset the current interval
    if current_sum >= interval:

        one_minute_intervals.append(current_interval)
        current_interval = []
        current_sum = 0
print(one_minute_intervals)

[[641.0, 633.0, 640.0, 657.0, 648.0, 649.0, 632.0, 633.0, 641.0, 625.0, 687.0, 672.0, 703.0, 672.0, 664.0, 695.0, 680.0, 718.0, 680.0, 703.0, 735.0, 711.0, 765.0, 805.0, 695.0, 742.0, 766.0, 789.0, 781.0, 727.0, 781.0, 758.0, 820.0, 813.0, 797.0, 742.0, 758.0, 750.0, 750.0, 742.0, 750.0, 719.0, 734.0, 719.0, 734.0, 719.0, 726.0, 727.0, 719.0, 672.0, 679.0, 688.0, 726.0, 743.0, 750.0, 664.0, 656.0, 664.0, 688.0, 742.0, 766.0, 773.0, 859.0, 852.0, 898.0, 914.0, 852.0, 852.0, 835.0, 829.0, 843.0, 828.0, 797.0, 774.0, 781.0, 766.0, 812.0, 727.0, 765.0, 719.0, 742.0, 766.0], [805.0, 835.0, 774.0, 711.0, 758.0, 765.0, 789.0, 797.0, 766.0, 758.0, 750.0, 726.0, 766.0, 750.0, 797.0, 758.0, 742.0, 781.0, 750.0, 805.0, 757.0, 782.0, 851.0, 836.0, 821.0, 789.0, 781.0, 805.0, 750.0, 789.0, 781.0, 765.0, 790.0, 789.0, 828.0, 781.0, 844.0, 797.0, 836.0, 812.0, 789.0, 774.0, 781.0, 781.0, 797.0, 742.0, 797.0, 742.0, 774.0, 796.0, 758.0, 813.0, 789.0, 773.0, 805.0, 734.0, 813.0, 750.0, 765.0, 797.0, 77

In [121]:
# Загрузка txt файла для SWELL
data = np.loadtxt("/content/drive/MyDrive/Colab Notebooks/Data_HRV/SWELL/p25.txt")
data = data[:, 1:]
data_list = data.flatten().tolist()
print(data_list)
# Define the time interval in milliseconds (1 minute = 60000 milliseconds)
interval = 60000

# Initialize variables
one_minute_intervals = []
current_interval = []
current_sum = 0

# # Iterate through the data
# for value in data_list:
#     # Add the current value to the current interval
#     current_interval.append(value)
#     current_sum += value
#     # If the sum of the current interval is greater than or equal to 1 minute, add it to the list of one-minute intervals and reset the current interval
#     if current_sum >= interval:
#         one_minute_intervals.append(current_interval)
#         current_interval = []
#         current_sum = 0

i = 0
while i < len(data_list):
    current_interval.append(sum(data_list[i : i+4])/4)
    current_sum += sum(data_list[i : i+4])/4
    if current_sum >= interval:

        one_minute_intervals.append(current_interval)
        current_interval = []
        current_sum = 0
    i += 4

[615.72266, 556.42315, 520.94738, 502.9983, 496.27891, 495.22932, 497.82694, 502.56323, 505.37215, 502.70924, 497.38237, 493.63937, 493.11357, 495.97944, 499.42942, 500.5989, 500.03138, 499.02674, 498.06834, 497.55505, 498.49449, 501.60585, 505.68829, 509.12532, 510.72517, 509.74715, 507.16237, 504.42175, 502.565, 502.25928, 502.84457, 503.31205, 503.42752, 503.22308, 502.59119, 501.44416, 500.17385, 499.15966, 497.4638, 493.81887, 488.17344, 480.94485, 473.45768, 470.83296, 476.06115, 482.89585, 486.05301, 486.64656, 486.73133, 488.3609, 493.52788, 504.07696, 518.92201, 532.83582, 543.1135, 552.62656, 563.64299, 573.94772, 580.53097, 582.30318, 579.21215, 571.21588, 558.29322, 541.72575, 524.84434, 511.48388, 506.1596, 510.02139, 516.28218, 519.4353, 520.54428, 521.75626, 525.09672, 531.52403, 537.06203, 537.11634, 533.46545, 530.52194, 529.43724, 529.23532, 528.92069, 527.5595, 525.72178, 525.72929, 527.80359, 528.74982, 527.22098, 526.43342, 528.78519, 532.64037, 536.03479, 538.6678

**Анализ полученной последовательности**

In [157]:
for count, interval in enumerate(one_minute_intervals):
    # print(len(interval))
    RR = remove_outliers(rr_intervals=interval, low_rri=300, high_rri=2000)
    rr = []
    for i in range(1, len(RR)):
      rr.append(2*(RR[i]-RR[i-1])/(RR[i]+RR[i-1]))
    time_1 = get_time_domain_features(RR)
    time_rr = get_time_domain_features(rr)
    freq_1 = get_frequency_domain_features(RR)
    pointcare_1 = get_poincare_plot_features(RR)
    sampen=get_sampen(RR)
    df = pd.DataFrame([(time_1['mean_nni'], time_1['median_nni'], time_1['sdnn'], time_1['rmssd'], time_1['sdsd'], time_1['sdnn']/time_1['rmssd'], time_1['mean_hr'], time_1['pnni_25'], time_1['pnni_50'], pointcare_1['sd1'], pointcare_1['sd2'], kurtosis(RR), skew(RR),
                        time_rr['mean_nni'], time_rr['median_nni'], time_rr['sdnn'], time_rr['rmssd'], time_rr['sdsd'], time_rr['sdnn']/time_rr['rmssd'], kurtosis(rr), skew(rr),
                        freq_1['vlf'], freq_1['vlf']/freq_1['total_power']*100, freq_1['lf'], freq_1['lf']/freq_1['total_power']*100, freq_1['lfnu'], freq_1['hf'], freq_1['hf']/freq_1['total_power']*100, freq_1['hfnu'], freq_1['total_power'], freq_1['lf_hf_ratio'], 1/freq_1['lf_hf_ratio'],
                        sampen['sampen'])],
                        columns=['MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD', 'SDRR_RMSSD', 'HR', 'pNN25', 'pNN50', 'SD1', 'SD2', 'KURT', 'SKEW',
                                 'MEAN_REL_RR', 'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR',
                                 'VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU', 'TP', 'LF_HF', 'HF_LF',
                                 'sampen'])
    df_end = df.loc[:, ['KURT', 'VLF', 'MEAN_REL_RR', 'SDSD_REL_RR', 'KURT_REL_RR', 'TP', 'MEDIAN_REL_RR', 'pNN25', 'LF', 'SD2', 'RMSSD_REL_RR', 'SDRR']]
    # print('A = %d, B = %d' % (3, 4))
    # print(pickled_model.predict(df_end))
    print('\n')
    print(pickled_model.predict(df_end)[0])
    if pickled_model.predict(df_end) == 1:
        print('During %d minute subject was in stress' % (count+1))
    else:
        print('During %d minute subject was not in stress' % (count+1))
        for i in interval:
          print(round(i))
    print('\n')



0 outlier(s) have been deleted.


1
During 1 minute subject was in stress


0 outlier(s) have been deleted.


0


<ipython-input-55-6280f053b022>:27: RuntimeWarning: divide by zero encountered in divide
  heart_rate_list = np.divide(60000, nn_intervals)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 239, using nperseg = 239
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-55-6280f053b022>:27: RuntimeWarning: divide by zero encountered in divide
  heart_rate_list = np.divide(60000, nn_intervals)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 239, using nperseg = 239
  warnings.warn('nperseg = {0:

During 2 minute subject was not in stress
844
875
797
719
750
750
719
797
734
766
813
797
781
797
766
797
797
750
828
797
813
766
734
734
875
875
750
781
813
781
781
797
828
781
844
844
766
781
766
766
797
828
781
797
828
813
797
766
797
844
688
719
734
766
703
734
828
719
797
781
797
766
766
781
734
734
750
750
719
719
781
750
734
625
750
641
859
844


0 outlier(s) have been deleted.


1
During 3 minute subject was in stress


0 outlier(s) have been deleted.


1


<ipython-input-55-6280f053b022>:27: RuntimeWarning: divide by zero encountered in divide
  heart_rate_list = np.divide(60000, nn_intervals)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 237, using nperseg = 237
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-55-6280f053b022>:27: RuntimeWarning: divide by zero encountered in divide
  heart_rate_list = np.divide(60000, nn_intervals)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 239, using nperseg = 239
  warnings.warn('nperseg = {0:

During 4 minute subject was in stress


0 outlier(s) have been deleted.


1
During 5 minute subject was in stress


0 outlier(s) have been deleted.




...

During 44 minute subject was not in stress
891
906
750
953
875
891
813
797
813
859
922
891
922
906
906
844
766
969
922
922
844
750
1031
953
969
891
891
938
891
953
844
875
813
828
906
953
953
984
953
906
828
844
906
938
953
813
844
844
906
891
875
906
922
844
844
844
891
906
875
891
891
906
844
828
859
938
984
891


0 outlier(s) have been deleted.


0
During 45 minute subject was not in stress
938
969
938
844
844
891
953
922
875
922
969
938
906
922
922
766
1000
953
938
875
859
922
984
906
797
1109
1000
891
859
891
906
875
906
953
953
938
875
922
938
938
875
844
922
969
953
906
953
969
984
984
1016
1047
1016
1000
953
891
891
922
953
906
953
1016
1000
922
906


0 outlier(s) have been deleted.




<ipython-input-55-6280f053b022>:27: RuntimeWarning: divide by zero encountered in divide
  heart_rate_list = np.divide(60000, nn_intervals)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 239, using nperseg = 239
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-55-6280f053b022>:27: RuntimeWarning: divide by zero encountered in divide
  heart_rate_list = np.divide(60000, nn_intervals)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 237, using nperseg = 237
  warnings.warn('nperseg = {0:

1
During 46 minute subject was in stress


0 outlier(s) have been deleted.


0
During 47 minute subject was not in stress
891
859
1000
922
906
984
953
922
938
938
859
844
891
969
891
859
938
969
906
859
875
891
828
859
1016
1000
1000
1125
1078
969
1047
1125
984
938
1016
1047
969
1063
1109
1031
969
1000
1109
891
859
719
891
891
984
844
1078
969
953
969
984
1109
906
1000
875
969
938
969
906


0 outlier(s) have been deleted.




<ipython-input-55-6280f053b022>:27: RuntimeWarning: divide by zero encountered in divide
  heart_rate_list = np.divide(60000, nn_intervals)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 237, using nperseg = 237
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-55-6280f053b022>:27: RuntimeWarning: divide by zero encountered in divide
  heart_rate_list = np.divide(60000, nn_intervals)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 241, using nperseg = 241
  warnings.warn('nperseg = {0:

1
During 48 minute subject was in stress




In [147]:
for i in one_minute_intervals:
  for j in i:
    print(round(j))
  print('\n')

766
891
781
813
781
703
734
844
750
797
734
859
672
641
703
688
734
828
766
766
766
734
609
766
828
719
641
656
594
609
656
719
797
781
719
766
734
719
672
766
797
859
859
859
828
797
750
781
750
797
797
844
797
766
813
750
766
719
750
781
797
859
828
766
750
797
828
828
797
797
734
766
750
703
734
734
797
891
859


844
875
797
719
750
750
719
797
734
766
813
797
781
797
766
797
797
750
828
797
813
766
734
734
875
875
750
781
813
781
781
797
828
781
844
844
766
781
766
766
797
828
781
797
828
813
797
766
797
844
688
719
734
766
703
734
828
719
797
781
797
766
766
781
734
734
750
750
719
719
781
750
734
625
750
641
859
844


875
766
766
734
813
859
719
766
766
813
766
859
891
844
859
859
844
781
766
734
703
703
766
734
859
813
734
797
813
766
797
875
891
906
781
734
781
750
719
734
734
781
797
844
875
828
813
797
719
734
703
703
781
813
844
859
828
859
844
813
828
766
750
734
641
766
750
688
797
766
766
688
734
672
672
672
672


688
719
781
875
781
859
844
734
797
766
734
781
781
813
78